In [1]:
import pandas as pd
import ast
import numpy as np
from src.Classifier import Classifier
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import ParameterGrid
from TextUtils import clean_article_url
from flatten_dict import flatten

ImportError: cannot import name 'normalize_hyphenated_words' from 'textacy.preprocessing' (/Users/mzavarzin/PyCharmProjects/Clean/ASFTE/venv/lib/python3.8/site-packages/textacy/preprocessing/__init__.py)

In [2]:
#Read corpus generated by the Corpus class
corpus_df = pd.read_csv('../data/corpus/corpus_augmented_backtranslation.csv')

In [25]:
#Different parameters for cross-validation and hyperparameter tuning
vector_params_lucene_bm25={"tf_type":"bm25","apply_idf":True,"idf_type":"smooth","apply_dl":True,"dl_type":"linear"} #BM25 weighting
vector_params_lucene_tfidf={"tf_type":"linear","apply_idf":True,"idf_type":"smooth","apply_dl":True,"dl_type":"linear"} #TF-IDF weighting

#Grid of parameters used for parameter tuning of the classifier
param_grid = {
    'gram': [1,2,3], #Different term length parameters
    'threshold': [0.1,0.15,0.2,0.25], #Different threshold parameters
    'vector_params': [vector_params_lucene_tfidf, vector_params_lucene_bm25],
    'filter_meta': [None, {"synthetic": 0}],
    'max_doc_depth': [2,3,4,5,6,None]
}

param_settings = list(ParameterGrid(param_grid))


In [3]:
#For evaluation of the classifier read manually labeled data
label = pd.read_csv('../data/labelledarticles/LabeledArticles.csv',delimiter=';')
label['formats/url'] = label['formats/url'].apply(lambda x: clean_article_url(x))
labels_dict = label[['formats/url','expected_product_hierarchy_aggr']].to_dict(orient='records')
label_values = [{dic['formats/url']: dic['expected_product_hierarchy_aggr']} for dic in labels_dict]
label_lookup = {}
for d in label_values:
    label_lookup.update(d)

NameError: name 'clean_article_url' is not defined

In [4]:
#Method to get the precision, recall, f1-scores for each run
def get_metrics(classified_mx, label_lookup):

    # Create the y_true column to compare classifier results with manually labeled results
    classified_mx['y_true'] = classified_mx.index
    classified_mx = classified_mx.replace({'y_true': label_lookup})

    #Fill missing rows (and rows below classification threshold) with Unlabeled
    classified_mx['y_true'] = classified_mx['y_true'].fillna('Unlabeled')
    classified_mx['y_pred'] = classified_mx['y_pred'].fillna('Unlabeled')
    classified_mx['y_true'] = classified_mx['y_true'].astype(str)
    classified_mx['y_pred'] = classified_mx['y_pred'].astype(str)

    #Encode writtern labels to ids
    le = preprocessing.LabelEncoder()
    le.fit(np.concatenate((classified_mx['y_true'].values,classified_mx['y_pred'].values)))

    #Apply encoding to labels
    classified_mx['true']=le.transform(classified_mx['y_true'])
    classified_mx['pred']=le.transform(classified_mx['y_pred'])
    classification_metrics = metrics.classification_report(classified_mx['true'], classified_mx['pred'],target_names=le.classes_,output_dict=True)
    aggregate_metrics = {key: classification_metrics[key] for key in ['accuracy','macro avg','weighted avg']}
    return flatten(aggregate_metrics, reducer='underscore')

In [5]:
#Initialize the classifier
cl = Classifier(corpus_df)


NameError: name 'Classifier' is not defined

In [7]:
import warnings
warnings.filterwarnings('ignore')


classified_df = cl.classify(**param_settings[0])
x = get_metrics(classified_df,label_lookup)

NameError: name 'cl' is not defined

In [30]:
#Perform grid search for best parameters
i=1
grid_search = pd.DataFrame()
for param in param_settings:

    run_details = {'__run':i,'_params':str(param)}
    classified_row = cl.classify(**param)
    row_metrics = get_metrics(classified_row,label_lookup)
    run_details.update(row_metrics)
    grid_search= grid_search.append(run_details,ignore_index=True)
    print(i)
    i+=1

# seed terms: 1934
# help article terms: 879
# common terms: 647
1
# seed terms: 1934
# help article terms: 879
# common terms: 647
2
# seed terms: 1934
# help article terms: 879
# common terms: 647
3
# seed terms: 1934
# help article terms: 879
# common terms: 647
4
# seed terms: 1934
# help article terms: 879
# common terms: 647
5
# seed terms: 1934
# help article terms: 879
# common terms: 647
6
# seed terms: 1934
# help article terms: 879
# common terms: 647
7
# seed terms: 1934
# help article terms: 879
# common terms: 647
8
# seed terms: 1934
# help article terms: 2439
# common terms: 1262
9
# seed terms: 1934
# help article terms: 2439
# common terms: 1262
10
# seed terms: 1934
# help article terms: 2439
# common terms: 1262
11
# seed terms: 1934
# help article terms: 2439
# common terms: 1262
12
# seed terms: 1934
# help article terms: 2439
# common terms: 1262
13
# seed terms: 1934
# help article terms: 2439
# common terms: 1262
14
# seed terms: 1934
# help article terms: 2439

In [18]:
#Save grid search results to csv
grid_search.to_csv('../data/evaluation/grid_search.csv')

In [19]:
#Select best grid search run by macro average f1-score
best_run = grid_search.iloc[grid_search['macro avg_f1-score'].idxmax()]
best_run

__run                                                                 127.0
_params                   {'filter_meta': None, 'gram': 3, 'max_doc_dept...
accuracy                                                           0.692308
macro avg_f1-score                                                 0.701096
macro avg_precision                                                0.764801
macro avg_recall                                                   0.699644
macro avg_support                                                      78.0
weighted avg_f1-score                                               0.69687
weighted avg_precision                                             0.832403
weighted avg_recall                                                0.692308
weighted avg_support                                                   78.0
Name: 126, dtype: object

In [22]:
best_param = dict(best_run).get('_params')
print('Best parameters: '+ best_param)

Best parameters: {'filter_meta': None, 'gram': 3, 'max_doc_depth': 5, 'threshold': 0.25, 'vector_params': {'tf_type': 'linear', 'apply_idf': True, 'idf_type': 'smooth', 'apply_dl': True, 'dl_type': 'linear'}}


In [38]:
#Classify documents with best parameter
#TODO: https://mediatum.ub.tum.de/doc/1545591/1545591.pdf + https://towardsdatascience.com/accuracy-recall-precision-f-score-specificity-which-to-optimize-on-867d3f11124
best_classification_df = cl.classify(**ast.literal_eval(best_param))
# Replace nan for unlabaled with 'Unlabeled' string
best_classification_df['y_pred'] = best_classification_df['y_pred'].fillna('Unlabeled')
best_classification_df['y_pred'] = best_classification_df['y_pred'].astype(str)
#Save best classification results to csv
best_classification_df.to_csv('../data/evaluation/best_classiciation.csv')


# seed terms: 33692
# help article terms: 302767
# common terms: 6157
